In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='3'

In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
import shutil
import seaborn as sns
import sklearn, os, time, h5py
import SimpleITK as sitk
import matplotlib.pyplot as plt
from random import randint
import random
from tensorflow import keras
from tensorflow.keras import Input, Model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Dropout, LeakyReLU
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, train_test_split
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from math import floor
from tensorflow.keras.applications import InceptionV3, DenseNet121
from bayes_opt import BayesianOptimization
import warnings
warnings.filterwarnings('ignore')

In [3]:
root_path = '/data1/registration/anhi/d1/'
import zipfile
with zipfile.ZipFile(root_path + "Dataset.zip","r") as zip_ref:
    zip_ref.extractall(root_path + "")

datasetFolderName = root_path + 'Dataset'
sourceFiles = []
classLabels = ['light', 'moderate', 'no', 'severe']
X = []
Y = []

img_rows, img_cols = 224, 224 
train_path = datasetFolderName+'/train/'
validation_path = datasetFolderName+'/valid/'
test_path = datasetFolderName+'/test/'

def transferBetweenFolders(source, dest, splitRate): 
    global sourceFiles
    sourceFiles = os.listdir(source)
    if(len(sourceFiles)!=0):
        transferFileNumbers = int(len(sourceFiles)*splitRate)
        transferIndex = random.sample(range(0, len(sourceFiles)), transferFileNumbers)
        for eachIndex in transferIndex:
            shutil.move(source+str(sourceFiles[eachIndex]), dest+str(sourceFiles[eachIndex]))
    else:
        print("No file moved. Source empty!")
        
def transferAllClassBetweenFolders(source, dest, splitRate):
    for label in classLabels:
        transferBetweenFolders(datasetFolderName+'/'+source+'/'+label+'/', 
                               datasetFolderName+'/'+dest+'/'+label+'/', splitRate)
                               
def my_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1Score = f1_score(y_true, y_pred, average='weighted') 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("recall : {}".format(recall))
    print("f1Score : {}".format(f1Score))
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, recall, f1Score

def prepareNameWithLabels(folderName):
    sourceFiles=os.listdir(datasetFolderName+'/train/'+folderName)
    for val in sourceFiles:
        X.append(val)
        for i in range(len(classLabels)):
            if(folderName==classLabels[i]):
                Y.append(i)
for i in range(len(classLabels)):
    prepareNameWithLabels(classLabels[i])
    
X = np.asarray(X)
Y = np.asarray(Y)
IMAGE_SIZE = [224, 224]

In [4]:
# Create function
def nn_cl_bo(neurons1, neurons2, activation1,activation2, optimizer, learning_rate, epochs, batch_size, dropout_rate, 
             layers1, layers2, dropout):
    optimizerL = ['SGD', 'Adam', 'RMSprop']
    optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate), 'RMSprop':RMSprop(lr=learning_rate)}               
    activationL = ['relu', 'sigmoid', 'selu', 'tanh', 'elu']

    neurons1 = round(neurons1)  
    neurons2 = round(neurons2)
    activation1 = activationL[round(activation1)]
    activation2 = activationL[round(activation2)]
    optimizer = (optimizerD[optimizerL[round(optimizer)]])
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def CNN_model():
        mobilenet = MobileNet(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
        for layer in mobilenet.layers:
            layer.trainable = False
        x = Flatten()(mobilenet.output)
        
        for i in range(layers1):
            x = Dense(neurons1, activation = activation1)(x)
        if dropout > 0.5:
            x = Dropout(dropout_rate)(x)
        for i in range(layers2):
            x = Dense(neurons2, activation = activation2)(x)

        prediction = Dense(len(classLabels), activation='softmax')(x)
        model = Model(inputs=mobilenet.input, outputs=prediction)
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])       
        return model

    skf = StratifiedKFold(n_splits=5, shuffle=True)
    skf.get_n_splits(X, Y)
    foldNum = 0
    scores = []
    for train_index, val_index in skf.split(X, Y):
        transferAllClassBetweenFolders('valid', 'train', 1.0)
        foldNum += 1
        X_train, X_val = X[train_index], X[val_index]
        Y_train, Y_val = Y[train_index], Y[val_index]
        for eachIndex in range(len(X_val)):
            classLabel = ''
            for i in range(len(classLabels)):
                if(Y_val[eachIndex]==i):
                    classLabel=classLabels[i]
            shutil.move(datasetFolderName+'/train/'+classLabel+'/'+X_val[eachIndex], 
                        datasetFolderName+'/valid/'+classLabel+'/'+X_val[eachIndex])   
            
        train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
        validation_datagen = ImageDataGenerator(rescale=1./255)
        test_datagen = ImageDataGenerator(rescale=1./255)

        train_generator = train_datagen.flow_from_directory(
            train_path,target_size=(img_rows, img_cols),
            batch_size=batch_size, class_mode='categorical', subset='training')     

        validation_generator = validation_datagen.flow_from_directory(
            validation_path,  target_size=(img_rows, img_cols),
            batch_size=batch_size, class_mode='categorical')   

        test_generator = test_datagen.flow_from_directory(
            test_path, target_size=(img_rows, img_cols),
            batch_size=batch_size,  class_mode=None, shuffle=False) 
    
    
        model1=CNN_model()
        history=model1.fit_generator(train_generator, validation_data=validation_generator, epochs=epochs)

        predictions = model1.predict(test_generator, verbose=1)
        yPredictions = np.argmax(predictions, axis=1)
        true_classes = test_generator.classes
        score = accuracy_score(true_classes, yPredictions)
        scores.append(score)
    
    avg_score = np.mean(scores)
    return avg_score

In [5]:
import time
start=time.time()
# Set paramaters
params_nn ={
    'neurons1': (64, 512),
    'neurons2': (64, 512),
    'activation1':(0, 4),
    'activation2':(0, 4),
    'optimizer':(0,2),
    'learning_rate':(1e-4, 1e-2),
    'batch_size':(8 ,64),
    'epochs':(30,70),
    'layers1':(1,2),
    'layers2':(1,2),
    'dropout': (0,1),
    'dropout_rate': (0, 0.3)
}
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo, params_nn, random_state=111)
#n_iter: how many steps of Bayesian optimization you want to perform. 
#The more step the more likely to find a good maximum you are
#init_points: How many teps of random exploration you want to perform. 
#Random exploration can help by diversifying the exploration space. 
nn_bo.maximize(init_points=10 , n_iter=4)
print('It takes %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | activa... | batch_... |  dropout  | dropou... |  epochs   |  layers1  |  layers2  | learni... | neurons1  | neurons2  | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
Found 1600 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
Found 180 images belonging to 4 classes.


2022-09-28 22:07:34.259782: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-28 22:07:34.961467: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10083 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:af:00.0, compute capability: 8.6


Epoch 1/36


2022-09-28 22:07:38.413926: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8401


 1/50 [..............................] - ETA: 3:20 - loss: 1.8591 - accuracy: 0.1562

...

Epoch 32/59
28/28 [==============================] - 17s 604ms/step - loss: 1.3591 - accuracy: 0.3363 - val_loss: 1.3491 - val_accuracy: 0.3500
Epoch 33/59
28/28 [==============================] - 17s 615ms/step - loss: 1.3557 - accuracy: 0.3269 - val_loss: 1.3543 - val_accuracy: 0.3500
Epoch 34/59
28/28 [==============================] - 17s 611ms/step - loss: 1.3559 - accuracy: 0.3500 - val_loss: 1.3531 - val_accuracy: 0.3500
Epoch 35/59
28/28 [==============================] - 17s 606ms/step - loss: 1.3607 - accuracy: 0.3206 - val_loss: 1.3612 - val_accuracy: 0.3500
Epoch 36/59
28/28 [==============================] - 17s 606ms/step - loss: 1.3847 - accuracy: 0.3125 - val_loss: 1.3473 - val_accuracy: 0.3500
Epoch 37/59
28/28 [==============================] - 17s 609ms/step - loss: 1.3623 - accuracy: 0.3388 - val_loss: 1.3628 - val_accuracy: 0.3500
Epoch 38/59
28/28 [==============================] - 17s 602ms/step - loss: 1.3605 - accuracy: 0.3256 - val_loss: 1.3551 - val_accuracy:

Epoch 29/59
28/28 [==============================] - 17s 608ms/step - loss: 1.3548 - accuracy: 0.3338 - val_loss: 1.3513 - val_accuracy: 0.3500
Epoch 30/59
28/28 [==============================] - 18s 629ms/step - loss: 1.3577 - accuracy: 0.3325 - val_loss: 1.3577 - val_accuracy: 0.2500
Epoch 31/59
28/28 [==============================] - 17s 616ms/step - loss: 1.3654 - accuracy: 0.3100 - val_loss: 1.3532 - val_accuracy: 0.3500
Epoch 32/59
28/28 [==============================] - 17s 613ms/step - loss: 1.3782 - accuracy: 0.3356 - val_loss: 1.4288 - val_accuracy: 0.2500
Epoch 33/59
28/28 [==============================] - 18s 628ms/step - loss: 1.3779 - accuracy: 0.3262 - val_loss: 1.3809 - val_accuracy: 0.3500
Epoch 34/59
28/28 [==============================] - 17s 614ms/step - loss: 1.3702 - accuracy: 0.3444 - val_loss: 1.3854 - val_accuracy: 0.2500
Epoch 35/59
28/28 [==============================] - 17s 608ms/step - loss: 1.3751 - accuracy: 0.3119 - val_loss: 1.3473 - val_accuracy:

Epoch 26/59
28/28 [==============================] - 17s 617ms/step - loss: 1.3674 - accuracy: 0.3281 - val_loss: 1.3629 - val_accuracy: 0.2500
Epoch 27/59
28/28 [==============================] - 17s 619ms/step - loss: 1.3610 - accuracy: 0.3244 - val_loss: 1.3576 - val_accuracy: 0.3500
Epoch 28/59
28/28 [==============================] - 17s 617ms/step - loss: 1.3604 - accuracy: 0.3331 - val_loss: 1.3691 - val_accuracy: 0.3500
Epoch 29/59
28/28 [==============================] - 17s 608ms/step - loss: 1.3685 - accuracy: 0.3262 - val_loss: 1.3566 - val_accuracy: 0.3500
Epoch 30/59
28/28 [==============================] - 17s 614ms/step - loss: 1.3604 - accuracy: 0.3281 - val_loss: 1.3523 - val_accuracy: 0.3500
Epoch 31/59
28/28 [==============================] - 17s 608ms/step - loss: 1.3553 - accuracy: 0.3344 - val_loss: 1.3511 - val_accuracy: 0.3500
Epoch 32/59
28/28 [==============================] - 17s 610ms/step - loss: 1.3513 - accuracy: 0.3300 - val_loss: 1.3607 - val_accuracy:

Epoch 23/59
28/28 [==============================] - 17s 605ms/step - loss: 1.3629 - accuracy: 0.3250 - val_loss: 1.3661 - val_accuracy: 0.2500
Epoch 24/59
28/28 [==============================] - 17s 599ms/step - loss: 1.3786 - accuracy: 0.2825 - val_loss: 1.3777 - val_accuracy: 0.2500
Epoch 25/59
28/28 [==============================] - 17s 617ms/step - loss: 1.3674 - accuracy: 0.3175 - val_loss: 1.3585 - val_accuracy: 0.3500
Epoch 26/59
28/28 [==============================] - 17s 609ms/step - loss: 1.3625 - accuracy: 0.3306 - val_loss: 1.3526 - val_accuracy: 0.3500
Epoch 27/59
28/28 [==============================] - 17s 602ms/step - loss: 1.3536 - accuracy: 0.3431 - val_loss: 1.3489 - val_accuracy: 0.3500
Epoch 28/59
28/28 [==============================] - 17s 598ms/step - loss: 1.3580 - accuracy: 0.3431 - val_loss: 1.3565 - val_accuracy: 0.2500
Epoch 29/59
28/28 [==============================] - 17s 604ms/step - loss: 1.3661 - accuracy: 0.3031 - val_loss: 1.3836 - val_accuracy:

Epoch 20/59
28/28 [==============================] - 18s 631ms/step - loss: 1.3609 - accuracy: 0.3175 - val_loss: 1.3600 - val_accuracy: 0.2500
Epoch 21/59
28/28 [==============================] - 18s 642ms/step - loss: 1.3686 - accuracy: 0.3113 - val_loss: 1.3569 - val_accuracy: 0.3500
Epoch 22/59
28/28 [==============================] - 18s 635ms/step - loss: 1.3611 - accuracy: 0.3431 - val_loss: 1.3791 - val_accuracy: 0.2500
Epoch 23/59
28/28 [==============================] - 18s 642ms/step - loss: 1.3688 - accuracy: 0.3269 - val_loss: 1.3809 - val_accuracy: 0.2500
Epoch 24/59
28/28 [==============================] - 18s 652ms/step - loss: 1.3755 - accuracy: 0.3069 - val_loss: 1.3716 - val_accuracy: 0.3500
Epoch 25/59
28/28 [==============================] - 17s 618ms/step - loss: 1.3689 - accuracy: 0.3156 - val_loss: 1.3741 - val_accuracy: 0.3500
Epoch 26/59
28/28 [==============================] - 18s 638ms/step - loss: 1.3683 - accuracy: 0.3237 - val_loss: 1.3563 - val_accuracy:

In [6]:
print(nn_bo.max)

{'target': 0.9388888888888889, 'params': {'activation1': 4.0, 'activation2': 0.0, 'batch_size': 64.0, 'dropout': 1.0, 'dropout_rate': 0.0, 'epochs': 70.0, 'layers1': 1.060728041596126, 'layers2': 1.0, 'learning_rate': 0.0001, 'neurons1': 187.4653394241566, 'neurons2': 480.4307459831931, 'optimizer': 2.0}}


In [8]:
params_nn_ = nn_bo.max['params']
learning_rate = params_nn_['learning_rate']
activationL = ['relu', 'sigmoid', 'selu', 'tanh', 'elu']
params_nn_['activation1'] = activationL[round(params_nn_['activation1'])]
params_nn_['activation2'] = activationL[round(params_nn_['activation2'])]
params_nn_['batch_size'] = round(params_nn_['batch_size'])
params_nn_['epochs'] = round(params_nn_['epochs'])
params_nn_['layers1'] = round(params_nn_['layers1'])
params_nn_['layers2'] = round(params_nn_['layers2'])
params_nn_['neurons1'] = round(params_nn_['neurons1'])
params_nn_['neurons2'] = round(params_nn_['neurons2'])
optimizerL = ['Adam', 'SGD', 'RMSprop']
optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate), 'RMSprop':RMSprop(lr=learning_rate)}           
params_nn_['optimizer'] = optimizerD[optimizerL[round(params_nn_['optimizer'])]]
params_nn_

{'activation1': 'elu',
 'activation2': 'relu',
 'batch_size': 64,
 'dropout': 1.0,
 'dropout_rate': 0.0,
 'epochs': 70,
 'layers1': 1,
 'layers2': 1,
 'learning_rate': 0.0001,
 'neurons1': 187,
 'neurons2': 480,
 'optimizer': <keras.optimizer_v2.rmsprop.RMSprop at 0x7f40185b1ed0>}

In [9]:
def CNN_model():
    mobilenet = MobileNet(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
    for layer in mobilenet.layers:
        layer.trainable = False
    x = Flatten()(mobilenet.output)

    for i in range(params_nn_['layers1']):
        x = Dense(params_nn_['neurons1'], activation = params_nn_['activation1'])(x)
    if params_nn_['dropout'] > 0.5:
        x = Dropout(params_nn_['dropout_rate'])(x)
    for i in range(params_nn_['layers2']):
        x = Dense(params_nn_['neurons2'], activation = params_nn_['activation2'])(x)

    prediction = Dense(len(classLabels), activation='softmax')(x)
    model = Model(inputs=mobilenet.input, outputs=prediction)
#     opt = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=params_nn_['optimizer'], metrics=['accuracy'])       
    return model

In [10]:
# skf = StratifiedKFold(n_splits=5, shuffle=True)
# skf.get_n_splits(X, Y)
# foldNum = 0
# for train_index, val_index in skf.split(X, Y):
#     transferAllClassBetweenFolders('valid', 'train', 1.0)
#     foldNum += 1
#     X_train, X_val = X[train_index], X[val_index]
#     Y_train, Y_val = Y[train_index], Y[val_index]
#     for eachIndex in range(len(X_val)):
#         classLabel = ''
#         for i in range(len(classLabels)):
#             if(Y_val[eachIndex]==i):
#                 classLabel=classLabels[i]
#         shutil.move(datasetFolderName+'/train/'+classLabel+'/'+X_val[eachIndex], 
#                     datasetFolderName+'/valid/'+classLabel+'/'+X_val[eachIndex])   

train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,target_size=(img_rows, img_cols),
    batch_size=params_nn_['batch_size'], class_mode='categorical', subset='training')     

validation_generator = validation_datagen.flow_from_directory(
    validation_path,  target_size=(img_rows, img_cols),
    batch_size=params_nn_['batch_size'], class_mode='categorical')   

test_generator = test_datagen.flow_from_directory(
    test_path, target_size=(img_rows, img_cols),
    batch_size=params_nn_['batch_size'],  class_mode=None, shuffle=False) 


model1=CNN_model()
history=model1.fit_generator(train_generator, validation_data=validation_generator, epochs=params_nn_['epochs'])

print("==============TEST RESULTS============") 
predictions = model1.predict(test_generator, verbose=1)
yPredictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

testAcc, testPrec, testRecall, testFScore = my_metrics(true_classes, yPredictions)

Found 1600 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
Found 180 images belonging to 4 classes.
Epoch 1/70
25/25 [==============================] - 20s 745ms/step - loss: 1.8101 - accuracy: 0.4363 - val_loss: 0.9155 - val_accuracy: 0.6025
Epoch 2/70
25/25 [==============================] - 17s 687ms/step - loss: 1.0145 - accuracy: 0.5694 - val_loss: 0.8829 - val_accuracy: 0.5900
Epoch 3/70
25/25 [==============================] - 17s 688ms/step - loss: 0.7778 - accuracy: 0.6837 - val_loss: 0.8585 - val_accuracy: 0.6375
Epoch 4/70
25/25 [==============================] - 18s 708ms/step - loss: 0.7184 - accuracy: 0.7131 - val_loss: 0.7891 - val_accuracy: 0.6700
Epoch 5/70
25/25 [==============================] - 18s 723ms/step - loss: 0.5522 - accuracy: 0.7806 - val_loss: 1.1654 - val_accuracy: 0.6250
Epoch 6/70
25/25 [==============================] - 18s 714ms/step - loss: 0.4504 - accuracy: 0.8194 - val_loss: 0.8307 - val_accuracy: 0.6850
Epoch 7/70
25/25 [

25/25 [==============================] - 17s 684ms/step - loss: 0.0079 - accuracy: 0.9981 - val_loss: 1.6703 - val_accuracy: 0.7425
Epoch 58/70
25/25 [==============================] - 17s 689ms/step - loss: 0.0079 - accuracy: 0.9987 - val_loss: 1.7345 - val_accuracy: 0.7275
Epoch 59/70
25/25 [==============================] - 17s 690ms/step - loss: 0.0091 - accuracy: 0.9975 - val_loss: 1.5465 - val_accuracy: 0.7475
Epoch 60/70
25/25 [==============================] - 18s 701ms/step - loss: 0.0075 - accuracy: 0.9994 - val_loss: 1.5906 - val_accuracy: 0.7700
Epoch 61/70
25/25 [==============================] - 17s 688ms/step - loss: 4.5119e-04 - accuracy: 1.0000 - val_loss: 1.6574 - val_accuracy: 0.7500
Epoch 62/70
25/25 [==============================] - 17s 676ms/step - loss: 0.0294 - accuracy: 0.9894 - val_loss: 1.8446 - val_accuracy: 0.7500
Epoch 63/70
25/25 [==============================] - 17s 688ms/step - loss: 0.0013 - accuracy: 0.9994 - val_loss: 1.5463 - val_accuracy: 0.7400


In [ ]:
#https://www.analyticsvidhya.com/blog/2021/05/bayesian-optimization-bayes_opt-or-hyperopt/
#https://machinelearningapplied.com/hyperparameter-search-with-bayesian-optimization-for-keras-cnn-classification-and-ensembling/
#https://towardsdatascience.com/tune-deep-neural-networks-using-bayesian-optimization-c9f6503a049f

In [ ]:
#https://towardsdatascience.com/tune-deep-neural-networks-using-bayesian-optimization-c9f6503a049f